In [ ]:
##!/usr/bin/env python
"""retrieve_advection_at_level.py

Purpose: Calculates the horizontal advection at a user defined level
Type: writes the time series to netcdf

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Wed Sep  4 09:52:50 AEST 2019

"""

In [1]:
# Load packages

import numpy as np
import netCDF4 as nc
import sys
import os
import xarray
import metpy.calc as mpcalc
import wrf


In [2]:
# Define Output locations

maindir = "/g/data/hh5/tmp/WRF-CABLE/AUS44/"
outdir = "/g/data/hh5/tmp/WRF-CABLE/AUS44/postproc_BS_EF/"

# Black Saturday
#sims = ["CTL_BS_30YR","EF_BS_1","EF_BS_2","EF_BS_3","EF_BS_4","EF_BS_5"]
#sims = ["NOSEED_EF_BS_1","NOSEED_EF_BS_2","NOSEED_EF_BS_3","NOSEED_EF_BS_4","NOSEED_EF_BS_5"]
#wrfoutfile = "wrfout_d01_2009-01-15_00:00:00"
#ntim = 649

# Brisbane Strongly Adiabatic Case
#sims = ["CTL_BNE_ADIABATIC","EF_BNE_ADIA_1","EF_BNE_ADIA_2","EF_BNE_ADIA_3","EF_BNE_ADIA_4","EF_BNE_ADIA_5"]
#wrfoutfile = "wrfout_d01_2002-11-17_00:00:00"
#ntim = 529

# Brisbane Diabatic Case
#sims = ["CTL_BNE_DIABATIC","EF_BNE_DIA_1","EF_BNE_DIA_2","EF_BNE_DIA_3","EF_BNE_DIA_4","EF_BNE_DIA_5"]
#wrfoutfile = "wrfout_d01_1997-12-02_00:00:00"
#ntim = 721

# Melbourne
#sims = ["CTL_MEL","EF_MEL_1","EF_MEL_2","EF_MEL_3","EF_MEL_4","EF_MEL_5"]
#wrfoutfile = "wrfout_d01_2000-01-26_00:00:00"
#ntim = 409

# Perth Case
sims = ["CTL_PERTH","EF_PERTH_1","EF_PERTH_2","EF_PERTH_3","EF_PERTH_4","EF_PERTH_5"]
wrfoutfile = "wrfout_d01_2000-12-11_00:00:00"
ntim = 505


In [3]:
# Load the mask file

datadir = "/g/data/hh5/tmp/WRF-CABLE/AUS44/CTL_BS_30YR/bdy_data/"
ldt_file = "%slis_input.d01.nc" %(datadir)
dataset = nc.Dataset(ldt_file,'r')
lsmask = dataset.variables['LANDMASK'][:,:]
lat2d = dataset.variables['lat'][:,:]
lon2d = dataset.variables['lon'][:,:]
# If the longitude spans [-180 180] then update to that instead it is [0 360] - better of AUS domain
if np.min(lon2d) < 0.0:
    lon2d = np.where(lon2d<0,lon2d+360,lon2d)
dataset.close()

lat1d = lat2d[:,0]
lon1d = lon2d[0,:]

nlat, nlon = lat2d.shape

# Use helper function defined above to calculate distance
# between lat/lon grid points
dx, dy = mpcalc.lat_lon_grid_deltas(lon1d, lat1d)

# Because of the way the data are returned we need a negative spacing. This
# will be easier in the next version of MetPy.
dy *= -1


Low-Level Heat Advection

In [4]:
# Loop through the experiments

for mind,mname in enumerate(sims):
    
    # Setup variable for output
    adv850 = np.empty((ntim,nlat,nlon),dtype=np.float64)
    mt850 = np.empty((ntim,nlat,nlon),dtype=np.float64)
    ua850 = np.empty((ntim,nlat,nlon),dtype=np.float64)
    va850 = np.empty((ntim,nlat,nlon),dtype=np.float64)
    wa850 = np.empty((ntim,nlat,nlon),dtype=np.float64)
    eth850 = np.empty((ntim,nlat,nlon),dtype=np.float64)
    omega850 = np.empty((ntim,nlat,nlon),dtype=np.float64)
    
    # Read in the required variables from the WRF output file
    file = "%s/%s/WRF_output/%s" %(maindir,mname,wrfoutfile)
    datafile = nc.Dataset(file,'r')
    ftimes = wrf.getvar(datafile,"times",timeidx=None)
    
    # Loop through the time indices
    for tind in range(ntim):

        pressure = wrf.getvar(datafile,"pressure",timeidx=tind,meta=False) #hPa
        temp = wrf.getvar(datafile,"tk",timeidx=tind,meta=False)
        qvapor = wrf.getvar(datafile,"QVAPOR",timeidx=tind,meta=False)
        uwind = wrf.getvar(datafile,"ua",timeidx=tind,meta=False)
        vwind = wrf.getvar(datafile,"va",timeidx=tind,meta=False)
        wwind = wrf.getvar(datafile,"wa",timeidx=tind,meta=False)
        eth = wrf.getvar(datafile,"theta_e",timeidx=tind,meta=False)
        omega = wrf.getvar(datafile,"omega",timeidx=tind,meta=False)

        # Extract variables at desired level
        ta = wrf.to_np(wrf.interplevel(temp,pressure,850.))
        qv = wrf.to_np(wrf.interplevel(qvapor,pressure,850.))
        ua = wrf.to_np(wrf.interplevel(uwind,pressure,850.))
        va = wrf.to_np(wrf.interplevel(vwind,pressure,850.))
        wa = wrf.to_np(wrf.interplevel(wwind,pressure,850.))
        etha = wrf.to_np(wrf.interplevel(eth,pressure,850.))
        omegaa = wrf.to_np(wrf.interplevel(omega,pressure,850.))
    
        # Calculate the advection
        adv850[tind,:,:] = mpcalc.advection(ta, [ua, va],(dx, dy), dim_order='yx')
        mt850[tind,:,:] = mpcalc.advection(qv, [ua, va],(dx, dy), dim_order='yx')
        ua850[tind,:,:] = ua
        va850[tind,:,:] = va
        wa850[tind,:,:] = wa
        eth850[tind,:,:] = etha
        omega850[tind,:,:] = omegaa
        
        del pressure, temp, qvapor, uwind, vwind, wwind, eth, omega, ta, qv, ua, va, wa, etha, omegaa

    datafile.close()

    # Write to netcdf
        
    # Define File
    dataset = nc.Dataset('%s/%s/wrfhrly_850hPa.nc' %(outdir,mname),'w')

    # Create dimensions
    latitude = dataset.createDimension('latitude',nlat)
    longitude = dataset.createDimension('longitude',nlon)
    time = dataset.createDimension('time',ntim)

    # Create coordinate variables
    lat = dataset.createVariable('lat2d',np.float64,('latitude','longitude',))
    lon = dataset.createVariable('lon2d',np.float64,('latitude','longitude',))
    times = dataset.createVariable('time',np.float64,('time',))

    # Create variables
    adv = dataset.createVariable('adv850', np.float64,('time','latitude','longitude'))
    mt = dataset.createVariable('mt850', np.float64,('time','latitude','longitude'))
    ua = dataset.createVariable('ua850', np.float64,('time','latitude','longitude'))
    va = dataset.createVariable('va850', np.float64,('time','latitude','longitude'))
    wa = dataset.createVariable('wa850', np.float64,('time','latitude','longitude'))
    eqpt = dataset.createVariable('eqpt', np.float64,('time','latitude','longitude')) # adding the equivalent potential temperature
    omega = dataset.createVariable('omega', np.float64,('time','latitude','longitude'))

    # Write data
    adv[:] = adv850[:]
    mt[:] = mt850[:]
    ua[:] = ua850[:]
    va[:] = va850[:]
    wa[:] = wa850[:]
    eqpt[:] = eth850[:]
    omega[:] = omega850[:]
    times[:] = ftimes[:]
    lat[:] = lat2d[:]
    lon[:] = lon2d[:]

    # Write the file
    dataset.close()
    
    # Cleanup
    del adv850, mt850, ua850, va850, wa850, eth850, omega850, ftimes
    

Surface Heat Advection

In [ ]:
# Loop through the experiments

for mind,mname in enumerate(sims):
    
    # Setup variable for output
    advs = np.empty((ntim,nlat,nlon),dtype=np.float64)
    uas = np.empty((ntim,nlat,nlon),dtype=np.float64)
    vas = np.empty((ntim,nlat,nlon),dtype=np.float64)
    
    # Read in the required variables from the WRF output file
    file = "%s/%s/WRF_output/%s" %(maindir,mname,wrfoutfile)
    datafile = nc.Dataset(file,'r')
    ftimes = wrf.getvar(datafile,"times",timeidx=None)
    
    # Loop through the time indices
    for tind in range(ntim):

        ta = datafile.variables['T2'][tind,:,:]
        ua = datafile.variables['U10'][tind,:,:]
        va = datafile.variables['V10'][tind,:,:]
    
        # Calculate the advection
        advs[tind,:,:] = mpcalc.advection(ta, [ua, va],(dx, dy), dim_order='yx')
        uas[tind,:,:] = ua
        vas[tind,:,:] = va
        
        del ta, ua, va

    datafile.close()

    # Write to netcdf
        
    # Define File
    dataset = nc.Dataset('%s/%s/wrfhrly_sfc.nc' %(outdir,mname),'w')

    # Create dimensions
    latitude = dataset.createDimension('latitude',nlat)
    longitude = dataset.createDimension('longitude',nlon)
    time = dataset.createDimension('time',ntim)

    # Create coordinate variables
    lat = dataset.createVariable('lat2d',np.float64,('latitude','longitude',))
    lon = dataset.createVariable('lon2d',np.float64,('latitude','longitude',))
    times = dataset.createVariable('time',np.float64,('time',))

    # Create variables
    adv = dataset.createVariable('adv', np.float64,('time','latitude','longitude'))
    ua = dataset.createVariable('uas', np.float64,('time','latitude','longitude'))
    va = dataset.createVariable('vas', np.float64,('time','latitude','longitude'))

    # Write data
    adv[:] = advs[:]
    ua[:] = uas[:]
    va[:] = vas[:]
    times[:] = ftimes[:]
    lat[:] = lat2d[:]
    lon[:] = lon2d[:]

    # Write the file
    dataset.close()
    
    # Cleanup
    del advs, uas, vas, ftimes
    